# **安装依赖库**

## **作者简介**

#### 作者：**⚡小闪电⚡**

#### B站主页
- [小闪电的B站主页](https://space.bilibili.com/122699831?spm_id_from=333.1007.0.0)

#### 交流群
欢迎加入AIDD交流群！  
加我微信（微信号: `xxxFLASHxxx`），邀请你进群。

## **安装依赖库**

In [ ]:
!pip install rdkit

# **第一部分：数据准备与预处理 (Data Preprocessing)**

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import os
import torch
import random
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from collections import Counter
from torch.utils.data import Dataset, DataLoader, random_split

# ==========================================
# 0. 全局配置与配色 (Visual Identity)
# ==========================================
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)

# 定义专属配色
COLOR_BLUE = (75 / 255, 116 / 255, 178 / 255) # 主色调 / 训练集 / 低甜
COLOR_RED = (219 / 255, 49 / 255, 36 / 255)   # 强调色 / 测试集 / 高甜
COLOR_GREY = "#f0f0f0"                        # 背景色

# 设置绘图风格
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.unicode_minus'] = False
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    plt.style.use('ggplot')

# ==========================================
# 1. 核心组件: SMILES Tokenizer
# ==========================================
class SMILESTokenizer:
    """
    基于化学正则表达式的分词器。
    支持保存/加载词表，保证推理时的一致性。
    """
    def __init__(self, max_len=128, vocab=None):
        self.max_len = max_len
        # 核心正则：识别 Br, Cl, [nH], =, #, 数字 等
        self.pattern =  r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
        self.regex = re.compile(self.pattern)

        # 特殊标记
        self.pad_token = "<pad>"
        self.unk_token = "<unk>"
        self.sos_token = "<sos>"
        self.eos_token = "<eos>"
        self.mask_token = "<mask>" # 预留给 BERT

        self.special_tokens = [self.pad_token, self.unk_token, self.sos_token, self.eos_token, self.mask_token]

        if vocab:
            self.vocab = vocab
        else:
            self.vocab = {token: i for i, token in enumerate(self.special_tokens)}

        self.inv_vocab = {i: token for token, i in self.vocab.items()}

    def tokenize(self, smiles):
        """正则切分"""
        return [token for token in self.regex.findall(smiles)]

    def build_vocab(self, smiles_list):
        """扫描整个数据集构建词表"""
        print("🏗️  正在构建化学词表...")
        counter = Counter()
        for s in smiles_list:
            tokens = self.tokenize(s)
            counter.update(tokens)

        # 按频率排序加入词表
        for token, count in counter.most_common():
            if token not in self.vocab:
                idx = len(self.vocab)
                self.vocab[token] = idx
                self.inv_vocab[idx] = token
        print(f"✅ 词表构建完成 | 大小: {len(self.vocab)} | Top Token: {counter.most_common(1)[0]}")

    def encode(self, smiles, add_special_tokens=True, padding=True, truncation=True):
        """SMILES -> IDs"""
        tokens = self.tokenize(smiles)

        # 1. 截断 (留位置给 SOS/EOS)
        if truncation and len(tokens) > self.max_len - 2:
            tokens = tokens[:self.max_len - 2]

        # 2. 查表
        ids = [self.vocab.get(t, self.vocab[self.unk_token]) for t in tokens]

        # 3. 添加特殊 Token
        if add_special_tokens:
            ids = [self.vocab[self.sos_token]] + ids + [self.vocab[self.eos_token]]

        # 4. 填充
        if padding:
            pad_len = self.max_len - len(ids)
            if pad_len > 0:
                ids = ids + [self.vocab[self.pad_token]] * pad_len

        return ids

    def decode(self, ids, skip_special_tokens=True):
        """IDs -> SMILES"""
        tokens = []
        for i in ids:
            token = self.inv_vocab.get(i, self.unk_token)
            if skip_special_tokens and token in self.special_tokens:
                continue
            tokens.append(token)
        return "".join(tokens)

    def save_vocab(self, path):
        with open(path, 'w') as f:
            json.dump(self.vocab, f, indent=2)

    @classmethod
    def load_vocab(cls, path, max_len=128):
        with open(path, 'r') as f:
            vocab = json.load(f)
        return cls(max_len=max_len, vocab=vocab)

# ==========================================
# 2. 数据管道 (Dataset & Split)
# ==========================================
class SweetnessDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.smiles = df['Smiles'].values
        self.labels = df['logSw'].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        s = self.smiles[idx]
        label = self.labels[idx]
        # 编码
        ids = self.tokenizer.encode(s)

        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.float),
            'raw_smiles': s # 保留原始字符串方便debug
        }

def prepare_data(csv_path, max_len=100, batch_size=32):
    # 1. 读取
    df = pd.read_csv(csv_path)
    # 列名清洗
    df.columns = [c.lower() for c in df.columns]
    rename_map = {'smiles': 'Smiles', 'logsw': 'logSw'}
    df.rename(columns=rename_map, inplace=True)
    df.dropna(subset=['Smiles', 'logSw'], inplace=True)

    # 2. Tokenizer
    tokenizer = SMILESTokenizer(max_len=max_len)
    tokenizer.build_vocab(df['Smiles'].tolist())

    # 3. 划分 (8:1:1)
    dataset = SweetnessDataset(df, tokenizer)
    total_len = len(dataset)
    train_len = int(0.8 * total_len)
    val_len = int(0.1 * total_len)
    test_len = total_len - train_len - val_len

    train_ds, val_ds, test_ds = random_split(
        dataset, [train_len, val_len, test_len],
        generator=torch.Generator().manual_seed(42)
    )

    # 4. Loaders
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)
    test_loader = DataLoader(test_ds, batch_size=batch_size)

    print(f"\n📊 数据划分完成:")
    print(f"   Train: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")

    return tokenizer, train_loader, val_loader, test_loader, df

# ==========================================
# 3. 花里胡哨的可视化 (Visualization)
# ==========================================
def visualize_data_distribution(df, tokenizer, max_len):
    """
    绘制仪表盘风格的数据分布图
    """
    # 计算统计量
    seq_lens = [len(tokenizer.tokenize(s)) for s in df['Smiles']]
    all_tokens = [t for s in df['Smiles'] for t in tokenizer.tokenize(s)]
    token_counts = Counter(all_tokens).most_common(10)

    # 创建画布
    fig = plt.figure(figsize=(15, 8), dpi=150)
    gs = gridspec.GridSpec(2, 2, width_ratios=[1.5, 1], height_ratios=[1, 1])
    fig.suptitle('Sweetness Dataset Analytics Dashboard', fontsize=18, fontweight='bold', y=0.95)

    # --- 图1: 序列长度分布 (左上) ---
    ax1 = plt.subplot(gs[0, 0])
    sns.histplot(seq_lens, bins=40, kde=True, color=COLOR_BLUE, edgecolor='white', alpha=0.7, ax=ax1)
    ax1.axvline(np.mean(seq_lens), color=COLOR_RED, linestyle='--', linewidth=2, label=f'Mean: {np.mean(seq_lens):.1f}')
    ax1.axvline(max_len, color='gray', linestyle=':', linewidth=2, label=f'Max Limit: {max_len}')
    ax1.set_title("Sequence Length Distribution", fontsize=12, fontweight='bold', loc='left', color='#333')
    ax1.legend()
    # 添加文字标注
    stats_text = f"Min: {min(seq_lens)}\nMax: {max(seq_lens)}\nMedian: {int(np.median(seq_lens))}"
    ax1.text(0.95, 0.5, stats_text, transform=ax1.transAxes, ha='right', bbox=dict(facecolor='white', alpha=0.8, boxstyle='round'))

    # --- 图2: Top 10 Token (右上) ---
    ax2 = plt.subplot(gs[:, 1]) # 占右边两行
    y = [x[0] for x in token_counts]
    x = [x[1] for x in token_counts]
    sns.barplot(x=x, y=y, palette=[COLOR_BLUE]*10, ax=ax2, orient='h')
    # 突出第一名
    ax2.patches[0].set_color(COLOR_RED)
    ax2.set_title("Top 10 Frequent Tokens", fontsize=12, fontweight='bold', loc='left')
    ax2.set_xlabel("Count")
    for i, v in enumerate(x):
        ax2.text(v + 3, i, str(v), color='black', va='center', fontsize=9)

    # --- 图3: 甜度值分布 (左下) ---
    ax3 = plt.subplot(gs[1, 0])
    # 绘制双色背景来区分高低甜度
    sns.kdeplot(df['logSw'], fill=True, color=COLOR_BLUE, alpha=0.3, ax=ax3, label="Density")
    sns.histplot(df['logSw'], bins=30, kde=False, stat="density", color=COLOR_BLUE, alpha=0.4, edgecolor='white', ax=ax3)

    # 绘制阈值线
    ax3.axvline(3.0, color=COLOR_RED, linestyle='--', linewidth=2)
    ax3.text(3.1, ax3.get_ylim()[1]*0.8, "High Sweetness (>3)", color=COLOR_RED, fontweight='bold')
    ax3.text(2.9, ax3.get_ylim()[1]*0.8, "Low Sweetness (<=3)", color=COLOR_BLUE, fontweight='bold', ha='right')

    ax3.set_title("Label Distribution (logSw)", fontsize=12, fontweight='bold', loc='left')
    ax3.set_xlabel("Log Sweetness")

    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    plt.show()

def visualize_tokenization_demo(tokenizer, smiles):
    """
    黑盒可视化：展示SMILES如何变成ID
    """
    tokens = tokenizer.tokenize(smiles)
    ids = tokenizer.encode(smiles)

    print("\n🔍 Tokenizer Deep Dive")
    print("="*60)
    print(f"🧪 Input:  {smiles}")
    print(f"✂️  Tokens: {tokens}")
    print("-" * 60)
    print(f"{'Token':<12} | {'ID':<6} | {'Visual'}")
    print("-" * 60)

    # 展示前5个和后5个，中间省略
    display_items = []
    # 重构显示列表 (含特殊token)
    full_tokens = [tokenizer.sos_token] + tokens + [tokenizer.eos_token]

    for i, t in enumerate(full_tokens):
        if i < 5 or i >= len(full_tokens)-3:
            # 简单的进度条可视化
            bar = "█" * (ids[i] % 10 + 1)
            print(f"{t:<12} | {ids[i]:<6} | {bar}")
        elif i == 5:
            print(f"{'...':<12} | {'...':<6} |")

    print("="*60)

# ==========================================
# 4. 执行入口
# ==========================================
if __name__ == "__main__":
    # 配置
    CSV_FILE = '/content/SweetpredDB.csv'
    MAX_LEN = 100
    BATCH_SIZE = 64

    # 1. 准备数据
    tokenizer, train_dl, val_dl, test_dl, raw_df = prepare_data(CSV_FILE, MAX_LEN, BATCH_SIZE)

    # 2. 保存词表 (供后续模型使用)
    tokenizer.save_vocab("vocab_sweetness.json")
    print(f"💾 词表已保存至 vocab_sweetness.json (Vocab Size: {len(tokenizer.vocab)})")

    # 3. 运行可视化
    print("\n🎨 正在绘制数据分布...")
    visualize_data_distribution(raw_df, tokenizer, MAX_LEN)

    # 4. 分词演示
    visualize_tokenization_demo(tokenizer, raw_df['Smiles'].iloc[0])

    # 5. 检查 Batch 数据
    sample = next(iter(train_dl))
    print("\n📦 Batch Data Check:")
    print(f"   Input Shape: {sample['input_ids'].shape}") # [64, 100]
    print(f"   Label Shape: {sample['labels'].shape}")    # [64]

# **第二部分：RNN 甜度预测 (Baseline Model)**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import time

# 复用之前的配色
COLOR_BLUE = (75 / 255, 116 / 255, 178 / 255)
COLOR_RED = (219 / 255, 49 / 255, 36 / 255)

# ==========================================
# 1. 模型定义: Bi-LSTM Regressor
# ==========================================
class BiLSTMRegressor(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=64, num_layers=2, pad_idx=0):
        super(BiLSTMRegressor, self).__init__()

        # 1. Embedding 层: 将离散的 ID 转换为稠密向量
        # padding_idx=pad_idx 很重要，它告诉 PyTorch 这个 ID 是填充的，不应该有梯度
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

        # 2. LSTM 层: 处理序列的核心
        # batch_first=True 让输入变成 [Batch, Seq, Feature] 符合直觉
        # bidirectional=True 开启双向，输出维度会变成 hidden_dim * 2
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.2 if num_layers > 1 else 0
        )

        # 3. 回归头 (MLP): 将提取的特征映射到甜度值
        # 输入维度是 hidden_dim * 2 (因为是双向)
        self.regressor = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1) # 输出一个标量：甜度值
        )

    def forward(self, x):
        # x shape: [Batch, Seq_Len]

        # [Step 1] Embedding
        # out: [Batch, Seq_Len, Embed_Dim]
        x_emb = self.embedding(x)

        # [Step 2] Bi-LSTM
        # out: [Batch, Seq_Len, Hidden_Dim * 2]
        # h_n, c_n 我们这里不需要，只需要所有时间步的输出
        lstm_out, _ = self.lstm(x_emb)

        # [Step 3] Mean Pooling (关键步骤)
        # 我们要把序列长度维度压缩掉，变成一个分子的整体向量
        # 简单的做法是取平均值: mean(dim=1)
        feature = lstm_out.mean(dim=1) # shape: [Batch, Hidden_Dim * 2]

        # [Step 4] MLP Regression
        # out: [Batch, 1]
        score = self.regressor(feature)

        return score.squeeze(-1) # 去掉最后一维，变成 [Batch]

# ==========================================
# 2. 训练引擎 (Training Engine)
# ==========================================
class Trainer:
    def __init__(self, model, device, lr=0.001):
        self.model = model.to(device)
        self.device = device
        self.optimizer = optim.Adam(model.parameters(), lr=lr)
        self.criterion = nn.MSELoss() # 回归任务标准损失函数

    def train_epoch(self, loader):
        self.model.train()
        total_loss = 0
        for batch in loader:
            # 搬运数据到 GPU/CPU
            ids = batch['input_ids'].to(self.device)
            labels = batch['labels'].to(self.device)

            self.optimizer.zero_grad()
            preds = self.model(ids)
            loss = self.criterion(preds, labels)
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()
        return total_loss / len(loader)

    def evaluate(self, loader):
        self.model.eval()
        all_preds = []
        all_labels = []
        total_loss = 0

        with torch.no_grad():
            for batch in loader:
                ids = batch['input_ids'].to(self.device)
                labels = batch['labels'].to(self.device)

                preds = self.model(ids)
                loss = self.criterion(preds, labels)
                total_loss += loss.item()

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_loss = total_loss / len(loader)
        r2 = r2_score(all_labels, all_preds)
        return avg_loss, r2, np.array(all_labels), np.array(all_preds)

# ==========================================
# 3. 运行训练与可视化
# ==========================================
def run_lstm_baseline(tokenizer, train_dl, val_dl, test_dl):
    print("\n🚀 启动 Baseline: Bi-LSTM 模型训练...")

    # 配置
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    EPOCHS = 20

    # 初始化模型
    # 注意：pad_idx 要从 tokenizer 的 vocab 中获取
    pad_id = tokenizer.vocab[tokenizer.pad_token]
    model = BiLSTMRegressor(vocab_size=len(tokenizer.vocab), pad_idx=pad_id)
    trainer = Trainer(model, DEVICE)

    # 记录日志
    history = {'train_loss': [], 'val_loss': [], 'val_r2': []}

    start_time = time.time()

    for epoch in range(EPOCHS):
        t_loss = trainer.train_epoch(train_dl)
        v_loss, v_r2, _, _ = trainer.evaluate(val_dl)

        history['train_loss'].append(t_loss)
        history['val_loss'].append(v_loss)
        history['val_r2'].append(v_r2)

        if (epoch + 1) % 1 == 0:
            print(f"Epoch {epoch+1:02d}/{EPOCHS} | Train Loss: {t_loss:.4f} | Val Loss: {v_loss:.4f} | Val R2: {v_r2:.4f}")

    print(f"✅ 训练完成! 耗时: {time.time()-start_time:.1f}s")

    # --- 最终评估 ---
    test_loss, test_r2, y_true, y_pred = trainer.evaluate(test_dl)
    print(f"\n🧪 测试集最终结果 -> MSE: {test_loss:.4f} | R2 Score: {test_r2:.4f}")

    # --- 可视化 ---
    plot_lstm_results(history, y_true, y_pred)

    return history

def plot_lstm_results(history, y_true, y_pred):
    plt.figure(figsize=(12, 5), dpi=100)

    # 图 1: 训练曲线 (Loss)
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss', color=COLOR_BLUE, linestyle='--')
    plt.plot(history['val_loss'], label='Val Loss', color=COLOR_BLUE, linewidth=2)
    plt.title('Bi-LSTM Learning Curve', fontsize=12, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 图 2: 真实值 vs 预测值
    plt.subplot(1, 2, 2)
    plt.scatter(y_true, y_pred, alpha=0.6, color=COLOR_BLUE, s=20, label='Test Samples')

    # 画对角线 (完美预测线)
    min_val = min(min(y_true), min(y_pred))
    max_val = max(max(y_true), max(y_pred))
    plt.plot([min_val, max_val], [min_val, max_val], color='gray', linestyle='--', label='Perfect Prediction')

    plt.title(f'Prediction Scatter (R2={r2_score(y_true, y_pred):.3f})', fontsize=12, fontweight='bold')
    plt.xlabel('True Sweetness')
    plt.ylabel('Predicted Sweetness')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# ==========================================
# 4. 执行入口
# ==========================================
if __name__ == "__main__":
    # 假设之前的变量 (tokenizer, train_dl...) 还在内存中
    # 如果没有，请先运行第一部分代码
    if 'tokenizer' in globals() and 'train_dl' in globals():
        lstm_history = run_lstm_baseline(tokenizer, train_dl, val_dl, test_dl)
    else:
        print("⚠️ 请先运行第一部分代码以准备数据！")

# **第三部分：Transformer 甜度预测 (Encoder Only)**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error  # 补全了 mean_squared_error

# 沿用之前的配色
COLOR_RED = (219 / 255, 49 / 255, 36 / 255)

# ==========================================
# 1. 核心组件: Positional Encoding
# ==========================================
class PositionalEncoding(nn.Module):
    """
    经典的 Sinusoidal 位置编码。
    注入位置信息，让 Transformer 知道 Token 的顺序。
    """
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        # 创建一个足够长的 PE 矩阵 [max_len, d_model]
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # 注册为 buffer (不是参数，不需要梯度更新，但随模型保存)
        self.register_buffer('pe', pe.unsqueeze(0)) # [1, max_len, d_model]

    def forward(self, x):
        # x: [Batch, Seq_Len, D_Model]
        # 切片取对应长度的位置编码相加
        x = x + self.pe[:, :x.size(1), :]
        return x

# ==========================================
# 2. 模型定义: Transformer Regressor
# ==========================================
class TransformerRegressor(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, dim_feedforward=256, pad_idx=0):
        super(TransformerRegressor, self).__init__()

        self.pad_idx = pad_idx

        # 1. Embedding
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)

        # 2. Positional Encoding
        self.pos_encoder = PositionalEncoding(d_model)

        # 3. Transformer Encoder 堆叠
        # batch_first=True 让输入保持 [Batch, Seq, Feature]
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True,
            dropout=0.1
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 4. 回归头 (MLP)
        self.regressor = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 1)
        )

        # 初始化参数 (对于 Transformer 很重要)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.regressor[0].bias.data.zero_()
        self.regressor[0].weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        # x: [Batch, Seq_Len]

        # 生成 Padding Mask (关键!)
        # src_key_padding_mask: [Batch, Seq_Len], True 表示是 padding，需要被忽略
        src_key_padding_mask = (x == self.pad_idx)

        # [Step 1] Embedding + Positional Encoding
        # out: [Batch, Seq, D_Model]
        x = self.embedding(x) * math.sqrt(x.size(-1)) # 缩放 embedding (Attention Is All You Need 原文操作)
        x = self.pos_encoder(x)

        # [Step 2] Transformer Encoder
        # out: [Batch, Seq, D_Model]
        x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)

        # [Step 3] Mean Pooling
        # 注意：这里粗暴平均包含了 padding 位置，严格来说应该 mask 掉再平均
        # 但由于 padding 的 embedding 通常是 0 或不更新，且有 mask 机制，影响较小，作为 Demo 可接受
        # 更好的做法是: (x * ~mask.unsqueeze(-1)).sum(1) / (~mask).sum(1).unsqueeze(-1)

        # 为了演示简单，我们取 mask 为 False (非 padding) 的 token 的平均值
        mask_expanded = (~src_key_padding_mask).unsqueeze(-1).float() # [Batch, Seq, 1]
        sum_embeddings = torch.sum(x * mask_expanded, dim=1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9) # 防止除零
        feature = sum_embeddings / sum_mask

        # [Step 4] MLP Regression
        score = self.regressor(feature)

        return score.squeeze(-1)

# ==========================================
# 3. 训练与可视化 (已添加保存功能)
# ==========================================
def run_transformer_challenger(tokenizer, train_dl, val_dl, test_dl):
    print("\n⚔️  启动 Challenger: Transformer 模型训练...")

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    EPOCHS = 20 # 保持和 RNN 一致以便对比

    # 获取 pad_id 用于 mask
    pad_id = tokenizer.vocab[tokenizer.pad_token]

    model = TransformerRegressor(
        vocab_size=len(tokenizer.vocab),
        pad_idx=pad_id,
        d_model=128,    # 保持维度规模相近
        nhead=4,        # 4头注意力
        num_layers=3    # 3层 Encoder
    ).to(DEVICE)

    # 定义优化器 (Transformer 通常需要更小的 lr 或者 warmup，这里先用标准 Adam)
    optimizer = optim.Adam(model.parameters(), lr=0.0005) # 比 RNN 稍微小一点的学习率更稳
    criterion = nn.MSELoss()

    history = {'train_loss': [], 'val_loss': [], 'val_r2': []}
    start_time = time.time()

    for epoch in range(EPOCHS):
        # --- Train Loop ---
        model.train()
        total_loss = 0
        for batch in train_dl:
            ids = batch['input_ids'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            optimizer.zero_grad()
            preds = model(ids)
            loss = criterion(preds, labels)
            loss.backward()

            # 梯度裁剪 (防止梯度爆炸，对 Transformer 很重要)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_dl)

        # --- Val Loop ---
        model.eval()
        val_preds, val_labels = [], []
        val_loss_sum = 0
        with torch.no_grad():
            for batch in val_dl:
                ids = batch['input_ids'].to(DEVICE)
                labels = batch['labels'].to(DEVICE)
                preds = model(ids)
                loss = criterion(preds, labels)
                val_loss_sum += loss.item()
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss_sum / len(val_dl)
        val_r2 = r2_score(val_labels, val_preds)

        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['val_r2'].append(val_r2)

        if (epoch + 1) % 1 == 0:
            print(f"Epoch {epoch+1:02d}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val R2: {val_r2:.4f}")

    print(f"✅ 训练完成! 耗时: {time.time()-start_time:.1f}s")

    # --- [新增] 保存 Checkpoint ---
    save_path = "transformer_sweetness.pt"
    torch.save(model.state_dict(), save_path)
    print(f"💾 模型参数已保存至: {save_path}")

    # --- Final Test ---
    model.eval()
    test_preds, test_labels = [], []
    with torch.no_grad():
        for batch in test_dl:
            ids = batch['input_ids'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            preds = model(ids)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    test_r2 = r2_score(test_labels, test_preds)
    test_mse = mean_squared_error(test_labels, test_preds)
    print(f"\n🧪 测试集最终结果 -> MSE: {test_mse:.4f} | R2 Score: {test_r2:.4f}")

    # --- Visualization ---
    plot_transformer_results(history, test_labels, test_preds)

    return history

def plot_transformer_results(history, y_true, y_pred):
    plt.figure(figsize=(12, 5), dpi=100)

    # 图 1: 训练曲线 (红色系)
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss', color=COLOR_RED, linestyle='--')
    plt.plot(history['val_loss'], label='Val Loss', color=COLOR_RED, linewidth=2)
    plt.title('Transformer Learning Curve', fontsize=12, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 图 2: 预测散点图
    plt.subplot(1, 2, 2)
    plt.scatter(y_true, y_pred, alpha=0.6, color=COLOR_RED, s=20, label='Test Samples')

    min_val = min(min(y_true), min(y_pred))
    max_val = max(max(y_true), max(y_pred))
    plt.plot([min_val, max_val], [min_val, max_val], color='gray', linestyle='--', label='Perfect Prediction')

    plt.title(f'Prediction Scatter (R2={r2_score(y_true, y_pred):.3f})', fontsize=12, fontweight='bold')
    plt.xlabel('True Sweetness')
    plt.ylabel('Predicted Sweetness')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# ==========================================
# 4. 执行入口
# ==========================================
if __name__ == "__main__":
    if 'tokenizer' in globals() and 'train_dl' in globals():
        transformer_history = run_transformer_challenger(tokenizer, train_dl, val_dl, test_dl)
    else:
        print("⚠️ 请先运行第一部分代码以准备数据！")

# **第四部分：巅峰对决 (Comparison)**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# 沿用之前的配色
COLOR_BLUE = (75 / 255, 116 / 255, 178 / 255) # Bi-LSTM
COLOR_RED = (219 / 255, 49 / 255, 36 / 255)   # Transformer

def visualize_peak_showdown(lstm_hist, transformer_hist):
    """
    绘制高大上的模型对比图
    """
    epochs = range(1, len(lstm_hist['val_loss']) + 1)

    # 创建画布
    fig = plt.figure(figsize=(16, 7), dpi=150)
    plt.subplots_adjust(wspace=0.25)

    # ==========================================
    # 1. Loss 曲线对比 (越低越好)
    # ==========================================
    ax1 = plt.subplot(1, 2, 1)

    # 绘制曲线
    l1 = ax1.plot(epochs, lstm_hist['val_loss'], 'o-', color=COLOR_BLUE, label='Bi-LSTM (Baseline)', linewidth=2, markersize=5)
    l2 = ax1.plot(epochs, transformer_hist['val_loss'], 's-', color=COLOR_RED, label='Transformer (Challenger)', linewidth=2, markersize=5)

    # 填充差距区域
    ax1.fill_between(epochs, lstm_hist['val_loss'], transformer_hist['val_loss'],
                     where=(np.array(transformer_hist['val_loss']) < np.array(lstm_hist['val_loss'])),
                     interpolate=True, color=COLOR_RED, alpha=0.1, label='Transformer Wins')
    ax1.fill_between(epochs, lstm_hist['val_loss'], transformer_hist['val_loss'],
                     where=(np.array(transformer_hist['val_loss']) >= np.array(lstm_hist['val_loss'])),
                     interpolate=True, color=COLOR_BLUE, alpha=0.1, label='RNN Wins')

    # 标注最小值
    min_lstm = min(lstm_hist['val_loss'])
    min_trans = min(transformer_hist['val_loss'])
    ax1.axhline(min_trans, color=COLOR_RED, linestyle=':', alpha=0.5)

    ax1.set_title("Validation Loss Comparison (Lower is Better)", fontsize=14, fontweight='bold')
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("MSE Loss")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # ==========================================
    # 2. R2 Score 曲线对比 (越高越好)
    # ==========================================
    ax2 = plt.subplot(1, 2, 2)

    ax2.plot(epochs, lstm_hist['val_r2'], 'o-', color=COLOR_BLUE, label='Bi-LSTM', linewidth=2)
    ax2.plot(epochs, transformer_hist['val_r2'], 's-', color=COLOR_RED, label='Transformer', linewidth=2)

    # 标注最佳点
    best_lstm_idx = np.argmax(lstm_hist['val_r2'])
    best_trans_idx = np.argmax(transformer_hist['val_r2'])

    ax2.annotate(f'Best LSTM: {lstm_hist["val_r2"][best_lstm_idx]:.3f}',
                 xy=(epochs[best_lstm_idx], lstm_hist['val_r2'][best_lstm_idx]),
                 xytext=(epochs[best_lstm_idx], lstm_hist['val_r2'][best_lstm_idx]-0.1),
                 arrowprops=dict(facecolor=COLOR_BLUE, shrink=0.05), color=COLOR_BLUE)

    ax2.annotate(f'Best Trans: {transformer_hist["val_r2"][best_trans_idx]:.3f}',
                 xy=(epochs[best_trans_idx], transformer_hist['val_r2'][best_trans_idx]),
                 xytext=(epochs[best_trans_idx], transformer_hist['val_r2'][best_trans_idx]-0.1),
                 arrowprops=dict(facecolor=COLOR_RED, shrink=0.05), color=COLOR_RED)

    ax2.set_title("Validation R2 Score Comparison (Higher is Better)", fontsize=14, fontweight='bold')
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("R2 Score")
    ax2.legend(loc='lower right')
    ax2.grid(True, alpha=0.3)

    plt.show()

def analyze_results(lstm_hist, trans_hist):
    """
    自动生成简单的分析报告
    """
    best_lstm = max(lstm_hist['val_r2'])
    best_trans = max(trans_hist['val_r2'])

    diff = best_trans - best_lstm

    print("\n🧐 结果深度分析 (Analysis Report)")
    print("=" * 60)
    print(f"🏆 最佳 Bi-LSTM R2:    {best_lstm:.4f}")
    print(f"🏆 最佳 Transformer R2: {best_trans:.4f}")
    print("-" * 60)

    if diff > 0.01:
        print("✅ 结论：Transformer 胜出！")
        print("   原因分析：")
        print("   1. 全局注意力机制 (Self-Attention) 成功捕捉了分子中远距离原子间的相互作用。")
        print("   2. 位置编码帮助模型理解了分子的拓扑结构顺序。")
    elif diff < -0.01:
        print("🔵 结论：Bi-LSTM 胜出（或差距不大）。")
        print("   原因分析：")
        print("   1. 数据集规模较小：Transformer 是数据饥渴型模型，在小样本上容易过拟合。")
        print("   2. 归纳偏置 (Inductive Bias)：RNN 天生适合处理序列，对于简单分子结构可能效率更高。")
        print("   3. 调参空间：Transformer 对超参数（学习率、Warmup）非常敏感，可能需要更精细的调优。")
    else:
        print("🤝 结论：两者旗鼓相当。")
        print("   这表明在该特定数据集和特征尺度下，序列建模的瓶颈可能不在于长距离依赖，")
        print("   或者当前的 Tokenizer 已经足够高效，让简单的 RNN 也能学得很好。")
    print("=" * 60)

# ==========================================
# 执行入口
# ==========================================
if __name__ == "__main__":
    # 确保之前的训练历史变量存在
    # lstm_history = ... (来自第二部分)
    # transformer_history = ... (来自第三部分)

    if 'lstm_history' in globals() and 'transformer_history' in globals():
        # 1. 绘制对比图
        visualize_peak_showdown(lstm_history, transformer_history)

        # 2. 输出分析文本
        analyze_results(lstm_history, transformer_history)
    else:
        print("⚠️ 缺少训练历史数据，请先运行第二和第三部分代码！")

# **第五部分：Transformer 架构大拓展 (The "Advanced" Part)**

## 1. Decoder-only (GPT-style): 无条件分子生成与条件控制分子生成

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================
# 1. MolGPT 模型定义 (Decoder-only)
# ==========================================
class MolGPT(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, max_len=128, pad_idx=0):
        super().__init__()
        self.d_model = d_model
        self.pad_idx = pad_idx

        # 1. Embedding & Positional Encoding
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoder = nn.Embedding(max_len, d_model) # GPT常用可学习的位置编码

        # 2. Transformer Decoder (GPT style)
        # 使用 TransformerEncoder 来实现 GPT，配合 causal mask 即可
        # batch_first=True
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=256, batch_first=True, dropout=0.1)
        self.transformer_decoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 3. Output Head
        self.fc_out = nn.Linear(d_model, vocab_size)

    def _generate_causal_mask(self, sz, device):
        """生成上三角掩码，防止偷看未来"""
        mask = (torch.triu(torch.ones(sz, sz, device=device)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, x):
        # x: [Batch, Seq_Len]
        seq_len = x.size(1)

        # Padding Mask (忽略 pad)
        src_key_padding_mask = (x == self.pad_idx)

        # Causal Mask (忽略未来)
        causal_mask = self._generate_causal_mask(seq_len, x.device)

        # Embedding + Position
        # 生成位置索引: [0, 1, 2, ..., seq_len-1]
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0).expand(x.size(0), -1)
        x = self.embedding(x) * math.sqrt(self.d_model) + self.pos_encoder(positions)

        # Transformer Forward
        # 注意：PyTorch 的 TransformerEncoder 允许传入 mask 做因果推断
        out = self.transformer_decoder(x, mask=causal_mask, src_key_padding_mask=src_key_padding_mask)

        # Logits: [Batch, Seq_Len, Vocab_Size]
        logits = self.fc_out(out)
        return logits

    @torch.no_grad()
    def generate(self, start_token_idx, max_new_tokens, tokenizer, temperature=1.0, stop_token_idx=None):
        """
        生成函数：给定起始 token，自动续写
        """
        self.eval()
        # idx: 当前生成的序列 [1, 1] (假设 batch=1)
        idx = torch.tensor([[start_token_idx]], dtype=torch.long, device=next(self.parameters()).device)

        for _ in range(max_new_tokens):
            # 截断到最大长度以免位置编码报错
            idx_cond = idx if idx.size(1) <= 100 else idx[:, -100:]

            # 前向传播
            logits = self(idx_cond)

            # 取最后一个时间步的预测
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)

            # 采样 (Multinomial Sampling)
            idx_next = torch.multinomial(probs, num_samples=1)

            # 拼接到序列中
            idx = torch.cat((idx, idx_next), dim=1)

            # 遇到 EOS 停止
            if stop_token_idx is not None and idx_next.item() == stop_token_idx:
                break

        return idx

### 无条件分子生成

In [ ]:
# ==========================================
# 任务 1: 无条件生成 (Unconditional Generation) - 修复版
# ==========================================

# 1. 准备生成任务的数据集
class GenDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.smiles = df['Smiles'].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        s = self.smiles[idx]
        # 编码: [SOS, t1, t2, ..., EOS]
        full_ids = self.tokenizer.encode(s)

        # 输入: [SOS, t1, t2, ..., tn] (去掉最后一个 EOS)
        # 标签: [t1, t2, ..., tn, EOS] (去掉第一个 SOS)
        # 这样 input[i] 预测的就是 label[i] (即 next token)
        inp = torch.tensor(full_ids[:-1], dtype=torch.long)
        tgt = torch.tensor(full_ids[1:], dtype=torch.long)

        return inp, tgt

def train_molgpt_unconditional(tokenizer, df):
    print("\n🧪 任务一：启动 MolGPT 无条件生成训练...")

    # --- 修复核心：collate_fn ---
    # 这里的 collate_fn 很重要，需要对 batch 进行动态 padding
    def collate_fn(batch):
        # 找出本 batch 最长序列
        max_len = max([len(item[0]) for item in batch])
        pad_id = tokenizer.vocab[tokenizer.pad_token]

        inps = []
        tgts = []
        for inp, tgt in batch:
            # Padding
            pad_len = max_len - len(inp)

            # 拼接
            # 注意：这里我们显式转换 padding tensor 为 long，防止自动推断为 float
            pad_tensor = torch.tensor([pad_id]*pad_len, dtype=torch.long)

            inps.append(torch.cat([inp, pad_tensor]))
            tgts.append(torch.cat([tgt, pad_tensor])) # target 也要 pad

        # 关键修复：在返回前强制转换为 .long()
        # 这样就能完美解决 "Expected tensor for argument #1 ... to have ... Long" 的报错
        return torch.stack(inps).long(), torch.stack(tgts).long()

    # Dataloader
    ds = GenDataset(df, tokenizer)
    dl = DataLoader(ds, batch_size=32, shuffle=True, collate_fn=collate_fn)

    # Init Model
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MolGPT(vocab_size=len(tokenizer.vocab), pad_idx=tokenizer.vocab[tokenizer.pad_token]).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.vocab[tokenizer.pad_token]) # 忽略 PAD 的 loss

    # Training Loop
    losses = []
    model.train()
    for epoch in range(30): # 训练 15 轮
        total_loss = 0
        for inp, tgt in dl:
            # 双重保险：送入 GPU 时再次确保是 long 类型
            inp, tgt = inp.to(DEVICE).long(), tgt.to(DEVICE).long()

            optimizer.zero_grad()
            logits = model(inp) # [Batch, Seq, Vocab]

            # Flatten for loss
            # logits: [Batch*Seq, Vocab], tgt: [Batch*Seq]
            loss = criterion(logits.view(-1, logits.size(-1)), tgt.view(-1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dl)
        losses.append(avg_loss)
        if (epoch+1) % 5 == 0:
            print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f}")

    # Demo Generation
    print("\n🎲 生成演示 (Random Generation):")
    sos_id = tokenizer.vocab[tokenizer.sos_token]
    eos_id = tokenizer.vocab[tokenizer.eos_token]

    generated_mols = []
    # 稍微生成多几个来看看效果
    for _ in range(5):
        try:
            gen_ids = model.generate(sos_id, max_new_tokens=50, tokenizer=tokenizer, stop_token_idx=eos_id)
            # decode 时跳过 special tokens
            smiles = tokenizer.decode(gen_ids[0].cpu().numpy(), skip_special_tokens=True)
            generated_mols.append(smiles)
            print(f"  -> {smiles}")
        except Exception as e:
            print(f"  -> 生成出错: {e}")

    return model, losses

# 执行
if 'tokenizer' in globals() and 'raw_df' in globals():
    gpt_model, gpt_losses = train_molgpt_unconditional(tokenizer, raw_df)

    # 画个简单的 Loss 曲线
    plt.figure(figsize=(6, 3))
    plt.plot(gpt_losses, color=COLOR_RED, marker='.')
    plt.title("MolGPT Training Loss")
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# 保存训练好的模型权重
torch.save(gpt_model.state_dict(), 'molgpt_unconditional.pt')
print("💾 模型已保存为 molgpt_unconditional.pt")

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from rdkit import Chem, rdBase
from rdkit.Chem import Draw, Descriptors, QED, AllChem, DataStructs
from tqdm import tqdm
import warnings

# ==========================================
# 0. 全局配置 (警告屏蔽 & 配色)
# ==========================================
# 1. 屏蔽 RDKit 的 "please use MorganGenerator" 刷屏警告
rdBase.DisableLog('rdApp.warning')
rdBase.DisableLog('rdApp.error')
warnings.filterwarnings('ignore')

# 2. 统一配色
COLOR_BLUE = (75 / 255, 116 / 255, 178 / 255)
COLOR_RED = (219 / 255, 49 / 255, 36 / 255)

# 3. 基础设置
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = 'molgpt_unconditional.pt'
GEN_NUM = 500  # 生成数量

# ==========================================
# 1. 加载模型与批量生成
# ==========================================
print("🚀 正在加载模型...")
# 确保 Tokenizer 已在内存中
if 'tokenizer' not in globals():
    raise ValueError("请先运行第一部分代码以初始化 tokenizer！")

pad_id = tokenizer.vocab[tokenizer.pad_token]
model = MolGPT(vocab_size=len(tokenizer.vocab), pad_idx=pad_id).to(DEVICE)

try:
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval()
    print("✅ 模型加载完成！")
except FileNotFoundError:
    print(f"❌ 未找到模型文件 {MODEL_PATH}，请先运行训练代码并保存模型。")

def generate_molecules(model, tokenizer, num_generate=100):
    generated_smiles = []
    sos_id = tokenizer.vocab[tokenizer.sos_token]
    eos_id = tokenizer.vocab[tokenizer.eos_token]

    print(f"\n🧪 开始生成 {num_generate} 个分子...")
    # 进度条
    for _ in tqdm(range(num_generate), desc="Generating"):
        try:
            gen_ids = model.generate(sos_id, max_new_tokens=100, tokenizer=tokenizer, stop_token_idx=eos_id)
            s = tokenizer.decode(gen_ids[0].cpu().numpy(), skip_special_tokens=True)
            generated_smiles.append(s)
        except Exception:
            continue
    return generated_smiles

# 执行生成
if 'model' in locals():
    gen_smiles_list = generate_molecules(model, tokenizer, num_generate=GEN_NUM)

    # ==========================================
    # 2. 合法性检查与性质计算
    # ==========================================
    print("\n⚗️ 正在进行化学性质计算与合法性检查...")
    valid_mols = []
    valid_smiles = []
    properties = {'QED': [], 'LogP': [], 'TPSA': []}

    for s in gen_smiles_list:
        mol = Chem.MolFromSmiles(s)
        if mol is not None:
            valid_mols.append(mol)
            valid_smiles.append(s)
            properties['QED'].append(QED.qed(mol))
            properties['LogP'].append(Descriptors.MolLogP(mol))
            properties['TPSA'].append(Descriptors.TPSA(mol))

    validity = len(valid_mols) / GEN_NUM
    print(f"✅ 生成结束：")
    print(f"   - 总尝试: {GEN_NUM}")
    print(f"   - 有效分子: {len(valid_mols)}")
    print(f"   - 有效率 (Validity): {validity:.2%}")

    # ==========================================
    # 3. 结果可视化 (严格使用指定配色)
    # ==========================================

    # --- 3.1 分子结构展示 ---
    print("\n🎨 绘制生成的分子结构示例...")
    if len(valid_mols) > 0:
        display_mols = valid_mols[:24] if len(valid_mols) >= 24 else valid_mols
        img = Draw.MolsToGridImage(display_mols, molsPerRow=6, subImgSize=(200, 200),
                                   legends=[f"Gen #{i+1}" for i in range(len(display_mols))])
        display(img)

    # --- 3.2 性质分布对比 (KDE Plot) ---
    print("\n📊 绘制性质分布对比图 (Train vs Generated)...")
    if len(valid_mols) > 0 and 'raw_df' in globals():
        # 准备训练集数据（采样）
        train_props = {'QED': [], 'LogP': [], 'TPSA': []}
        sample_df = raw_df.sample(n=min(1000, len(raw_df)), random_state=42)

        for s in sample_df['Smiles']:
            m = Chem.MolFromSmiles(s)
            if m:
                train_props['QED'].append(QED.qed(m))
                train_props['LogP'].append(Descriptors.MolLogP(m))
                train_props['TPSA'].append(Descriptors.TPSA(m))

        # 绘图
        fig, axes = plt.subplots(1, 3, figsize=(18, 5), dpi=120)

        def plot_dist(ax, train_data, gen_data, title):
            # 训练集用蓝色 (Base)
            sns.kdeplot(train_data, fill=True, color=COLOR_BLUE, label='Train Set', ax=ax, alpha=0.3, linewidth=0)
            # 生成集用红色 (Generated)
            sns.kdeplot(gen_data, fill=True, color=COLOR_RED, label='Generated', ax=ax, alpha=0.5, linewidth=2)
            ax.set_title(title, fontsize=12, fontweight='bold')
            ax.legend()
            ax.grid(True, alpha=0.2)

        plot_dist(axes[0], train_props['QED'], properties['QED'], 'QED Distribution')
        plot_dist(axes[1], train_props['LogP'], properties['LogP'], 'LogP Distribution')
        plot_dist(axes[2], train_props['TPSA'], properties['TPSA'], 'TPSA Distribution')

        plt.tight_layout()
        plt.show()

    # --- 3.3 相似度分布 (Novelty) ---
    print("\n🔍 计算与训练集的相似度分布...")
    if len(valid_mols) > 0 and 'raw_df' in globals():
        # 计算指纹
        gen_fps = [AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) for m in valid_mols]

        train_mols_sample = [Chem.MolFromSmiles(s) for s in sample_df['Smiles']]
        train_mols_sample = [m for m in train_mols_sample if m is not None]
        train_fps = [AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) for m in train_mols_sample]

        max_sims = []
        for g_fp in tqdm(gen_fps, desc="Computing Similarity"):
            if not train_fps: break
            sims = DataStructs.BulkTanimotoSimilarity(g_fp, train_fps)
            max_sims.append(max(sims) if sims else 0.0)

        plt.figure(figsize=(8, 5), dpi=120)
        # 使用红色绘制分布
        sns.histplot(max_sims, bins=30, kde=True, color=COLOR_RED, edgecolor='white', alpha=0.7)
        plt.axvline(np.mean(max_sims), color=COLOR_BLUE, linestyle='--', linewidth=2, label=f'Mean: {np.mean(max_sims):.2f}')

        plt.title("Max Similarity to Training Set (Novelty Check)", fontsize=12, fontweight='bold')
        plt.xlabel("Max Tanimoto Similarity")
        plt.ylabel("Count")
        plt.legend()
        plt.grid(True, alpha=0.2)
        plt.show()

### 条件控制分子生成

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 任务 2: 属性条件生成 (修复版)
# ==========================================

def train_conditional_molgpt_and_save(base_tokenizer, df):
    print("\n🎛️ 任务二：启动属性条件生成训练 (Ctrl+C for Molecules)...")

    # 1. 扩展 Tokenizer (Add Control Tags)
    cond_tokenizer = base_tokenizer

    # 检查是否已添加，避免重复添加
    if "<high>" not in cond_tokenizer.vocab:
        new_id_high = len(cond_tokenizer.vocab)
        cond_tokenizer.vocab["<high>"] = new_id_high
        cond_tokenizer.inv_vocab[new_id_high] = "<high>"

        new_id_low = len(cond_tokenizer.vocab)
        cond_tokenizer.vocab["<low>"] = new_id_low
        cond_tokenizer.inv_vocab[new_id_low] = "<low>"
        print(f"✅ 已添加控制符: <high> (ID: {new_id_high}), <low> (ID: {new_id_low})")

    # 保存扩展后的词表
    cond_tokenizer.save_vocab("smiles_vocab_conditional.json")

    # 2. 准备带条件的数据集 (核心修复处)
    class CondDataset(Dataset):
        def __init__(self, df, tokenizer):
            self.data = []
            self.tokenizer = tokenizer
            self.high_token = tokenizer.vocab["<high>"]
            self.low_token = tokenizer.vocab["<low>"]
            self.sos_token = tokenizer.vocab[tokenizer.sos_token]
            self.eos_token = tokenizer.vocab[tokenizer.eos_token]

            for _, row in df.iterrows():
                s = row['Smiles']
                val = row['logSw']

                # [关键修复]: padding=False
                # 我们只想要纯净的原子序列，不要在中间夹杂 pad
                base_ids = tokenizer.encode(s, add_special_tokens=False, padding=False, truncation=True)

                cond_token = self.high_token if val > 3.0 else self.low_token

                # 正确的结构: [SOS, COND, Atom1, Atom2, ..., EOS]
                full_ids = [self.sos_token, cond_token] + base_ids + [self.eos_token]
                self.data.append(full_ids)

        def __len__(self): return len(self.data)

        def __getitem__(self, idx):
            ids = self.data[idx]
            # 转换为 tensor
            return torch.tensor(ids[:-1], dtype=torch.long), torch.tensor(ids[1:], dtype=torch.long)

    # Collate Fn (在这里统一做 Padding)
    def collate_fn(batch):
        # 找出本 batch 最长序列
        max_len = max([len(item[0]) for item in batch])
        pad_id = cond_tokenizer.vocab[cond_tokenizer.pad_token]

        inps, tgts = [], []
        for inp, tgt in batch:
            pad_len = max_len - len(inp)
            pad_tensor = torch.tensor([pad_id] * pad_len, dtype=torch.long)

            # 拼接: 序列 + Pad
            inps.append(torch.cat([inp, pad_tensor]))
            tgts.append(torch.cat([tgt, pad_tensor]))

        return torch.stack(inps).long(), torch.stack(tgts).long()

    # 3. 训练流程
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 确保 pad_idx 正确传入
    pad_id = cond_tokenizer.vocab[cond_tokenizer.pad_token]
    model = MolGPT(vocab_size=len(cond_tokenizer.vocab), pad_idx=pad_id).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

    ds = CondDataset(df, cond_tokenizer)
    dl = DataLoader(ds, batch_size=32, shuffle=True, collate_fn=collate_fn)

    losses = []
    model.train()
    print("Training...", end="")

    EPOCHS = 30
    for epoch in range(EPOCHS):
        total_loss = 0
        for inp, tgt in dl:
            inp, tgt = inp.to(DEVICE), tgt.to(DEVICE)
            optimizer.zero_grad()
            logits = model(inp)
            loss = criterion(logits.view(-1, logits.size(-1)), tgt.view(-1))
            loss.backward()

            # [建议]: 加上梯度裁剪，防止梯度爆炸
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dl)
        losses.append(avg_loss)

        if (epoch+1) % 1 == 0:
            print(f" [Ep{epoch+1} L:{avg_loss:.4f}]", end="")

    print("\n✅ Training Done!")

    # 4. 保存模型
    torch.save(model.state_dict(), 'molgpt_conditional.pt')
    print("💾 条件生成模型已保存为 molgpt_conditional.pt")

    # 5. 绘制 Loss 曲线
    plt.figure(figsize=(8, 4), dpi=120)
    plt.plot(range(1, EPOCHS+1), losses, 'o-', color=COLOR_RED, linewidth=2, markersize=4)
    plt.title("Conditional MolGPT Training Loss", fontsize=12, fontweight='bold')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True, alpha=0.3)
    plt.show()

# 执行训练
if 'tokenizer' in globals() and 'raw_df' in globals():
    train_conditional_molgpt_and_save(tokenizer, raw_df)

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem

# ==========================================
# 0. 配置
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MOLGPT_PATH = 'molgpt_conditional.pt'
GEN_NUM = 50  # 修改为 50，确保最后每类都有 50 个有效分子

# ==========================================
# 1. 加载 MolGPT 生成模型
# ==========================================
print("🚀 [1/2] 正在加载 MolGPT 条件生成模型...")

# 确保 Tokenizer 包含控制符
if "<high>" not in tokenizer.vocab:
    raise ValueError("Tokenizer 缺少控制符，请确保之前的训练代码已运行！")

pad_id = tokenizer.vocab[tokenizer.pad_token]

# 初始化模型结构
model = MolGPT(vocab_size=len(tokenizer.vocab), pad_idx=pad_id).to(DEVICE)

try:
    model.load_state_dict(torch.load(MOLGPT_PATH, map_location=DEVICE))
    model.eval()
    print("✅ MolGPT 模型权重加载成功！")
except Exception as e:
    print(f"❌ MolGPT 模型加载失败: {e}")

# ==========================================
# 2. 生成过程 (Temperature + Top-K) - 修改版
# ==========================================
def generate_batch(target_tag, num_generate=50, temperature=0.8, top_k=5, max_attempts=1000):
    """
    修改逻辑：循环生成直到收集到 num_generate 个有效分子为止。
    max_attempts: 防止模型崩坏导致死循环的最大尝试次数
    """
    tag_id = tokenizer.vocab[target_tag]
    sos_id = tokenizer.vocab[tokenizer.sos_token]
    eos_id = tokenizer.vocab[tokenizer.eos_token]

    # 初始输入: [SOS, TAG]
    start_seq = torch.tensor([[sos_id, tag_id]], dtype=torch.long).to(DEVICE)
    generated_smiles = []

    print(f"\n🧬 正在生成 {num_generate} 个 {target_tag} 分子...")

    # 使用 tqdm 手动管理进度条
    pbar = tqdm(total=num_generate, desc=f"Generating {target_tag}")
    attempts = 0

    while len(generated_smiles) < num_generate:
        attempts += 1
        if attempts > max_attempts:
            print(f"\n⚠️ 警告：尝试次数超过 {max_attempts}，提前停止。")
            break

        try:
            idx = start_seq
            with torch.no_grad():
                for _ in range(100): # Max Len
                    logits = model(idx)
                    # Temperature Scaling
                    logits = logits[:, -1, :] / temperature

                    # Top-K Filtering
                    if top_k is not None:
                        v, _ = torch.topk(logits, top_k)
                        logits[logits < v[:, [-1]]] = -float('Inf')

                    probs = F.softmax(logits, dim=-1)
                    next_token = torch.multinomial(probs, 1)

                    if next_token.item() == eos_id:
                        break
                    idx = torch.cat([idx, next_token], dim=1)

            # Decode
            out_ids = idx[0].cpu().numpy()
            if len(out_ids) > 2:
                # 跳过 SOS 和 TAG
                smiles = tokenizer.decode(out_ids[2:], skip_special_tokens=True)
                # 检查合法性
                if Chem.MolFromSmiles(smiles):
                    generated_smiles.append(smiles)
                    pbar.update(1) # 有效分子+1，进度条更新
        except Exception:
            continue

    pbar.close()
    return generated_smiles

# 执行生成
high_smiles = generate_batch("<high>", num_generate=GEN_NUM, temperature=0.8)
low_smiles = generate_batch("<low>", num_generate=GEN_NUM, temperature=0.8)

print(f"\n✅ 生成统计: High Valid: {len(high_smiles)}, Low Valid: {len(low_smiles)}")

In [ ]:
import torch
import torch.nn as nn
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Draw
from torch.nn.utils.rnn import pad_sequence
from scipy import stats

# ==========================================
# 0. 配置与配色
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 这里直接指定好路径 ---
SWEETNESS_MODEL_PATH = '/content/transformer_sweetness.pt'

COLOR_BLUE = (75 / 255, 116 / 255, 178 / 255) # Low Sweetness
COLOR_RED = (219 / 255, 49 / 255, 36 / 255)   # High Sweetness

# ==========================================
# 1. 理化性质分布对比 (LogP, QED, TPSA)
# ==========================================
print("\n📊 [1/4] 计算并绘制理化性质分布...")
data = []

# 收集 High 数据
for s in high_smiles:
    mol = Chem.MolFromSmiles(s)
    if mol:
        data.append({
            "Type": "High Sweetness (Gen)",
            "LogP": Descriptors.MolLogP(mol),
            "QED": QED.qed(mol),
            "TPSA": Descriptors.TPSA(mol)
        })

# 收集 Low 数据
for s in low_smiles:
    mol = Chem.MolFromSmiles(s)
    if mol:
        data.append({
            "Type": "Low Sweetness (Gen)",
            "LogP": Descriptors.MolLogP(mol),
            "QED": QED.qed(mol),
            "TPSA": Descriptors.TPSA(mol)
        })

df_res = pd.DataFrame(data)

if not df_res.empty:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5), dpi=120)
    my_palette = {"High Sweetness (Gen)": COLOR_RED, "Low Sweetness (Gen)": COLOR_BLUE}

    sns.violinplot(data=df_res, x="Type", y="LogP", palette=my_palette, ax=axes[0])
    axes[0].set_title("LogP Distribution")

    sns.violinplot(data=df_res, x="Type", y="QED", palette=my_palette, ax=axes[1])
    axes[1].set_title("QED Distribution")

    sns.violinplot(data=df_res, x="Type", y="TPSA", palette=my_palette, ax=axes[2])
    axes[2].set_title("TPSA Distribution")

    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 没有足够的数据进行绘图")

# ==========================================
# 2. 定义预测模型结构 (用于加载权重)
# ==========================================
# 必须重新定义类，否则 pickle 加载会报错
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

class TransformerRegressor(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, dim_feedforward=256, pad_idx=0):
        super().__init__()
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.regressor = nn.Sequential(nn.Linear(d_model, 64), nn.ReLU(), nn.Dropout(0.1), nn.Linear(64, 1))

    def forward(self, x):
        src_key_padding_mask = (x == self.pad_idx)
        x = self.embedding(x) * math.sqrt(x.size(-1))
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)
        mask_expanded = (~src_key_padding_mask).unsqueeze(-1).float()
        sum_embeddings = torch.sum(x * mask_expanded, dim=1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        feature = sum_embeddings / sum_mask
        return self.regressor(feature).squeeze(-1)

# ==========================================
# 3. 甜度预测与验证可视化
# ==========================================
print("\n🧪 [2/4] 加载预测模型并评估生成分子...")

pad_id = tokenizer.vocab[tokenizer.pad_token]
sweet_predictor = TransformerRegressor(vocab_size=46, pad_idx=pad_id).to(DEVICE)
# sweet_predictor = TransformerRegressor(vocab_size=len(tokenizer.vocab), pad_idx=pad_id).to(DEVICE)

try:
    # 加载指定的 Checkpoint
    sweet_predictor.load_state_dict(torch.load(SWEETNESS_MODEL_PATH, map_location=DEVICE))
    sweet_predictor.eval()
    print(f"✅ 甜度预测模型加载成功: {SWEETNESS_MODEL_PATH}")

    # 定义预测函数
    def predict_sweetness(smiles_list):
        if not smiles_list: return []
        batch_ids = []
        for s in smiles_list:
            # 这里的 encode 默认行为应该与训练 Regressor 时一致 (包含 SOS/EOS)
            ids = tokenizer.encode(s)
            batch_ids.append(torch.tensor(ids, dtype=torch.long))

        inp = pad_sequence(batch_ids, batch_first=True, padding_value=pad_id).to(DEVICE)
        with torch.no_grad():
            preds = sweet_predictor(inp)
        return preds.cpu().numpy()

    # 执行预测
    print("   正在预测 High 组甜度...")
    pred_high = predict_sweetness(high_smiles)
    print("   正在预测 Low 组甜度...")
    pred_low = predict_sweetness(low_smiles)

    # 打印统计结果
    print(f"\n📈 统计结果:")
    print(f"   Avg Sweetness (High Gen): {np.mean(pred_high):.3f}")
    print(f"   Avg Sweetness (Low Gen):  {np.mean(pred_low):.3f}")

    # T-test 检验
    if len(pred_high) > 1 and len(pred_low) > 1:
        t_stat, p_val = stats.ttest_ind(pred_high, pred_low, equal_var=False)
        print(f"   T-test p-value: {p_val:.2e}")

    # ==========================================
    # 4. 保存数据到 CSV (新增功能)
    # ==========================================
    print("\n💾 [3/4] 正在保存生成数据到 CSV...")
    df_high = pd.DataFrame({
        'SMILES': high_smiles,
        'Predicted_LogSw': pred_high,
        'Type': 'High Sweetness'
    })

    df_low = pd.DataFrame({
        'SMILES': low_smiles,
        'Predicted_LogSw': pred_low,
        'Type': 'Low Sweetness'
    })

    df_final = pd.concat([df_high, df_low], ignore_index=True)
    csv_filename = 'generated_molecules_prediction.csv'
    df_final.to_csv(csv_filename, index=False)
    print(f"✅ 文件已保存: {csv_filename} (包含 {len(df_final)} 条记录)")

    # ==========================================
    # 5. 绘制甜度分布图
    # ==========================================
    print("\n🎨 [4/4] 绘制甜度分布与分子结构...")
    plt.figure(figsize=(10, 6), dpi=120)

    # 使用 KDE Plot 可视化分布差异
    sns.kdeplot(pred_high, fill=True, color=COLOR_RED, label=f'High Sweetness Gen (Mean:{np.mean(pred_high):.2f})', alpha=0.5)
    sns.kdeplot(pred_low, fill=True, color=COLOR_BLUE, label=f'Low Sweetness Gen (Mean:{np.mean(pred_low):.2f})', alpha=0.5)

    plt.axvline(3.0, color='gray', linestyle='--', label='Threshold (3.0)')
    plt.title("Predicted Sweetness Distribution: High vs Low Control", fontsize=14, fontweight='bold')
    plt.xlabel("Predicted Log Sweetness (logSw)")
    plt.ylabel("Density")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    # ==========================================
    # 6. 随机可视化分子结构 (新增功能)
    # ==========================================
    def visualize_random_samples(smiles_list, preds, title, n=10):
        # 如果数量不足 n，则全部显示
        if len(smiles_list) < n:
            n = len(smiles_list)

        # 随机采样索引
        indices = random.sample(range(len(smiles_list)), n)

        mols = []
        legends = []
        for idx in indices:
            s = smiles_list[idx]
            p = preds[idx]
            mol = Chem.MolFromSmiles(s)
            if mol:
                mols.append(mol)
                legends.append(f"Pred: {p:.2f}")

        if mols:
            img = Draw.MolsToGridImage(mols, molsPerRow=5, subImgSize=(200, 200), legends=legends, returnPNG=False)
            print(f"\n🖼️ {title} (Random {n} Samples)")
            display(img)

    # 可视化高甜分子
    visualize_random_samples(high_smiles, pred_high, "High Sweetness Generated Molecules")

    # 可视化低甜分子
    visualize_random_samples(low_smiles, pred_low, "Low Sweetness Generated Molecules")

except FileNotFoundError:
    print(f"❌ 未找到预测模型文件 {SWEETNESS_MODEL_PATH}，请确认路径正确或先运行 Transformer 训练代码。")
except Exception as e:
    print(f"❌ 预测过程出错: {e}")

## 2. Encoder-Decoder (Seq2Seq):分子优化

第一部分：数据预处理 (配对与构建数据集)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# ==========================================
# 1. 构造优化对 (Optimization Pairs)
# ==========================================
def construct_optimization_dataset(df):
    print("MATCHING: 正在构建分子优化对 (Low -> High)...")

    # 1. 分组
    low_df = df[df['logSw'] <= 3.0].reset_index(drop=True)
    high_df = df[df['logSw'] > 3.0].reset_index(drop=True)

    print(f"   - 低甜分子池 (Source): {len(low_df)}")
    print(f"   - 高甜分子池 (Target): {len(high_df)}")

    # 2. 计算指纹
    def get_fp(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol: return AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
        return None

    print("   - 计算分子指纹中...")
    low_fps = [get_fp(s) for s in low_df['Smiles']]
    high_fps = [get_fp(s) for s in high_df['Smiles']]

    # 3. 寻找最近邻 (Pairing)
    pairs = []
    # 过滤无效指纹
    valid_high_indices = [i for i, fp in enumerate(high_fps) if fp is not None]
    high_fps_valid = [high_fps[i] for i in valid_high_indices]

    for i, fp_query in enumerate(tqdm(low_fps, desc="Pairing Molecules")):
        if fp_query is None: continue

        # 计算当前低甜分子与所有高甜分子的相似度
        sims = DataStructs.BulkTanimotoSimilarity(fp_query, high_fps_valid)

        if not sims: continue

        # 找到最相似的那个高甜分子
        best_local_idx = np.argmax(sims)
        max_sim = sims[best_local_idx]

        # 筛选逻辑：
        # 放宽一点下限，保证有足够的数据训练
        if 0.35 < max_sim < 0.98:
            src_smiles = low_df.iloc[i]['Smiles']
            true_high_idx = valid_high_indices[best_local_idx]
            tgt_smiles = high_df.iloc[true_high_idx]['Smiles']
            pairs.append((src_smiles, tgt_smiles))

    print(f"✅ 配对完成: 找到 {len(pairs)} 对合理的优化路径")
    return pairs

# ==========================================
# 2. 数据集定义 (修复 Padding 问题)
# ==========================================
class OptimizationDataset(Dataset):
    def __init__(self, pairs, tokenizer):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.sos_id = tokenizer.vocab[tokenizer.sos_token]
        self.eos_id = tokenizer.vocab[tokenizer.eos_token]

    def __len__(self): return len(self.pairs)

    def __getitem__(self, idx):
        src_s, tgt_s = self.pairs[idx]

        # [关键修复]: padding=False
        # 确保 Dataset 返回的是纯净的 Token 序列，不包含 pad
        src_ids = self.tokenizer.encode(src_s, add_special_tokens=False, padding=False)
        tgt_ids = self.tokenizer.encode(tgt_s, add_special_tokens=False, padding=False)

        # 构建张量
        return {
            'src': torch.tensor(src_ids + [self.eos_id], dtype=torch.long),
            'tgt_in': torch.tensor([self.sos_id] + tgt_ids, dtype=torch.long),
            'tgt_out': torch.tensor(tgt_ids + [self.eos_id], dtype=torch.long)
        }

def collate_fn(batch):
    pad_id = tokenizer.vocab[tokenizer.pad_token]
    # 使用 pad_sequence 统一在 batch 级别进行 padding
    srcs = pad_sequence([x['src'] for x in batch], batch_first=True, padding_value=pad_id)
    tgt_ins = pad_sequence([x['tgt_in'] for x in batch], batch_first=True, padding_value=pad_id)
    tgt_outs = pad_sequence([x['tgt_out'] for x in batch], batch_first=True, padding_value=pad_id)
    return srcs, tgt_ins, tgt_outs

# --- 执行预处理 ---
if 'tokenizer' in globals() and 'raw_df' in globals():
    opt_pairs = construct_optimization_dataset(raw_df)
    # 划分训练集和测试集
    if len(opt_pairs) > 0:
        train_pairs, test_pairs = train_test_split(opt_pairs, test_size=0.1, random_state=42)

        train_ds = OptimizationDataset(train_pairs, tokenizer)
        train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
        print(f"📊 数据准备就绪: Train={len(train_pairs)}, Test={len(test_pairs)}")
    else:
        print("❌ 警告：未找到匹配的分子对，请检查相似度阈值或数据量。")

第二部分：模型训练与保存

In [ ]:
import torch.nn as nn
import torch.optim as optim
import math
import matplotlib.pyplot as plt

# ==========================================
# 3. Seq2Seq 模型架构
# ==========================================
class MolSeq2Seq(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, pad_idx=0):
        super().__init__()
        self.d_model = d_model
        self.pad_idx = pad_idx

        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        # 简单的位置编码 Embedding
        self.pos_encoder = nn.Embedding(1024, d_model)

        self.transformer = nn.Transformer(
            d_model=d_model, nhead=nhead,
            num_encoder_layers=num_layers, num_decoder_layers=num_layers,
            dim_feedforward=256,
            batch_first=True, dropout=0.1
        )
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt):
        # src: [Batch, Src_Len]
        # tgt: [Batch, Tgt_Len]

        # 1. Masks
        src_pad_mask = (src == self.pad_idx)
        tgt_pad_mask = (tgt == self.pad_idx)
        # Decoder 自回归掩码 (防止看未来)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1)).to(src.device)

        # 2. Embed + Pos
        src_seq = torch.arange(src.size(1), device=src.device).unsqueeze(0)
        tgt_seq = torch.arange(tgt.size(1), device=tgt.device).unsqueeze(0)

        # 限制位置索引范围，防止过长报错
        src_seq = src_seq.clamp(max=1023)
        tgt_seq = tgt_seq.clamp(max=1023)

        src_emb = self.embedding(src) * math.sqrt(self.d_model) + self.pos_encoder(src_seq)
        tgt_emb = self.embedding(tgt) * math.sqrt(self.d_model) + self.pos_encoder(tgt_seq)

        # 3. Transformer Forward
        out = self.transformer(
            src=src_emb, tgt=tgt_emb,
            src_key_padding_mask=src_pad_mask,
            tgt_key_padding_mask=tgt_pad_mask,
            memory_key_padding_mask=src_pad_mask,
            tgt_mask=tgt_mask
        )

        return self.fc_out(out)

# ==========================================
# 4. 训练循环
# ==========================================
def train_optimization_model():
    print("\n🚀 开始训练分子优化模型 (Seq2Seq)...")

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pad_id = tokenizer.vocab[tokenizer.pad_token]

    model = MolSeq2Seq(len(tokenizer.vocab), pad_idx=pad_id).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

    losses = []
    model.train()

    EPOCHS = 30
    for epoch in range(EPOCHS):
        total_loss = 0
        for src, tgt_in, tgt_out in train_dl:
            src, tgt_in, tgt_out = src.to(DEVICE), tgt_in.to(DEVICE), tgt_out.to(DEVICE)

            optimizer.zero_grad()
            logits = model(src, tgt_in)

            # Flatten for loss
            loss = criterion(logits.view(-1, logits.size(-1)), tgt_out.view(-1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_dl)
        losses.append(avg_loss)

        if (epoch+1) % 1 == 0:
            print(f"Epoch {epoch+1:02d} | Loss: {avg_loss:.4f}")

    print("✅ 训练完成")

    # 保存模型
    torch.save(model.state_dict(), 'mol_optimization.pt')
    print("💾 模型已保存为 mol_optimization.pt")

    # 绘制 Loss
    plt.figure(figsize=(6,3))
    plt.plot(losses, color='green', marker='o')
    plt.title("Optimization Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    return model

# --- 执行训练 ---
if 'train_dl' in globals():
    opt_model = train_optimization_model()

第三部分：模型推理、生成与结构可视化

In [ ]:
import torch
import torch.nn as nn
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from rdkit import Chem
from rdkit.Chem import Draw
from torch.nn.utils.rnn import pad_sequence

# ==========================================
# 0. 配置与类定义 (为了加载 Sweetness 模型)
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SWEETNESS_MODEL_PATH = '/content/transformer_sweetness.pt'
OPT_MODEL_PATH = 'mol_optimization.pt'

COLOR_BLUE = (75 / 255, 116 / 255, 178 / 255) # Low
COLOR_GREEN = (46 / 255, 139 / 255, 87 / 255) # Optimized

# --- 重复定义 TransformerRegressor 以便加载 Checkpoint ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

class TransformerRegressor(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, dim_feedforward=256, pad_idx=0):
        super().__init__()
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.regressor = nn.Sequential(nn.Linear(d_model, 64), nn.ReLU(), nn.Dropout(0.1), nn.Linear(64, 1))
    def forward(self, x):
        src_key_padding_mask = (x == self.pad_idx)
        x = self.embedding(x) * math.sqrt(x.size(-1))
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)
        mask_expanded = (~src_key_padding_mask).unsqueeze(-1).float()
        sum_embeddings = torch.sum(x * mask_expanded, dim=1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        feature = sum_embeddings / sum_mask
        return self.regressor(feature).squeeze(-1)

# ==========================================
# 1. 核心评估函数
# ==========================================
def evaluate_optimization(test_pairs, tokenizer, num_samples=100):
    print("\n🔍 正在进行分子优化推理与评估...")

    pad_id = tokenizer.vocab[tokenizer.pad_token]
    sos_id = tokenizer.vocab[tokenizer.sos_token]
    eos_id = tokenizer.vocab[tokenizer.eos_token]

    # --- 加载两个模型 ---
    # 1. 优化模型
    opt_model = MolSeq2Seq(len(tokenizer.vocab), pad_idx=pad_id).to(DEVICE)
    try:
        opt_model.load_state_dict(torch.load(OPT_MODEL_PATH, map_location=DEVICE))
        opt_model.eval()
    except FileNotFoundError:
        print(f"❌ 找不到优化模型 {OPT_MODEL_PATH}")
        return

    # 2. 甜度预测模型
    # sweet_model = TransformerRegressor(vocab_size=len(tokenizer.vocab), pad_idx=pad_id).to(DEVICE)
    sweet_model = TransformerRegressor(vocab_size=46, pad_idx=pad_id).to(DEVICE)
    try:
        sweet_model.load_state_dict(torch.load(SWEETNESS_MODEL_PATH, map_location=DEVICE))
        sweet_model.eval()
        print("✅ 成功加载: 优化模型 & 甜度预测模型")
    except FileNotFoundError:
        print(f"❌ 找不到甜度预测模型 {SWEETNESS_MODEL_PATH}")
        return

    # --- 辅助：甜度预测函数 ---
    def get_pred_score(smiles):
        if not smiles: return 0.0
        ids = tokenizer.encode(smiles)
        inp = torch.tensor([ids], dtype=torch.long).to(DEVICE)
        with torch.no_grad():
            score = sweet_model(inp)
        return score.item()

    # --- 开始推理 ---
    results = []

    # 如果测试集不够大，就全部使用
    eval_indices = range(len(test_pairs))
    if len(test_pairs) > num_samples:
        eval_indices = random.sample(range(len(test_pairs)), num_samples)

    print(f"   处理 {len(eval_indices)} 个样本...")

    for idx in eval_indices:
        src_s, tgt_s = test_pairs[idx] # src=低甜(Input), tgt=高甜(Ground Truth)

        # 1. 运行优化模型 (Source -> Optimized)
        src_ids = tokenizer.encode(src_s, add_special_tokens=False) + [eos_id]
        src_tensor = torch.tensor([src_ids], dtype=torch.long).to(DEVICE)
        curr_tgt = torch.tensor([[sos_id]], dtype=torch.long).to(DEVICE)

        # Greedy Decode
        with torch.no_grad():
            for _ in range(100):
                logits = opt_model(src_tensor, curr_tgt)
                next_token = logits[:, -1, :].argmax(dim=-1).unsqueeze(1)
                if next_token.item() == eos_id: break
                curr_tgt = torch.cat([curr_tgt, next_token], dim=1)

        pred_ids = curr_tgt[0].cpu().numpy()[1:]
        opt_s = tokenizer.decode(pred_ids, skip_special_tokens=True)

        # 2. 检查有效性 & 预测甜度
        mol_src = Chem.MolFromSmiles(src_s)
        mol_opt = Chem.MolFromSmiles(opt_s)

        if mol_src and mol_opt:
            score_src = get_pred_score(src_s)
            score_opt = get_pred_score(opt_s)

            results.append({
                'Source_SMILES': src_s,
                'Source_LogSw': score_src,
                'Optimized_SMILES': opt_s,
                'Optimized_LogSw': score_opt,
                'Improvement': score_opt - score_src,
                'Valid': True
            })
        else:
             results.append({
                'Source_SMILES': src_s,
                'Optimized_SMILES': opt_s,
                'Valid': False
            })

    # --- 保存结果 ---
    df_res = pd.DataFrame(results)
    df_valid = df_res[df_res['Valid'] == True]

    csv_path = 'optimization_results.csv'
    df_valid.to_csv(csv_path, index=False)
    print(f"✅ 结果已保存至 {csv_path} (有效样本: {len(df_valid)}/{len(df_res)})")

    if df_valid.empty:
        print("⚠️ 没有生成有效的优化分子，请检查模型训练情况。")
        return

    # --- 可视化 1: 甜度分布对比 ---
    plt.figure(figsize=(10, 6), dpi=120)
    sns.kdeplot(df_valid['Source_LogSw'], fill=True, color=COLOR_BLUE, label='Original (Low Sweet)', alpha=0.5)
    sns.kdeplot(df_valid['Optimized_LogSw'], fill=True, color=COLOR_GREEN, label='Optimized (AI Generated)', alpha=0.5)
    plt.axvline(3.0, color='gray', linestyle='--', label='Target Threshold (3.0)')
    plt.title("Sweetness Optimization Effect", fontsize=14, fontweight='bold')
    plt.xlabel("Predicted Log Sweetness")
    plt.legend()
    plt.show()

    print(f"   平均提升 (Improvement): {df_valid['Improvement'].mean():.3f}")

    # --- 可视化 2: 随机展示 5 对分子 ---
    print("\n🖼️ 分子结构对比 (Input -> Optimized)")
    sample_df = df_valid.sample(min(5, len(df_valid)))

    mols = []
    legends = []
    for _, row in sample_df.iterrows():
        mols.append(Chem.MolFromSmiles(row['Source_SMILES']))
        mols.append(Chem.MolFromSmiles(row['Optimized_SMILES']))
        legends.append(f"Src: {row['Source_LogSw']:.2f}")
        legends.append(f"Opt: {row['Optimized_LogSw']:.2f}")

    img = Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(250, 250), legends=legends)
    display(img)

# --- 执行 ---
if 'test_pairs' in globals():
    evaluate_optimization(test_pairs, tokenizer, num_samples=200)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F # 引入 F 用于 softmax
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Draw
from torch.nn.utils.rnn import pad_sequence

# ==========================================
# 0. 配置与模型类定义 (必须重新定义以加载 Checkpoint)
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SWEETNESS_MODEL_PATH = '/content/transformer_sweetness.pt'
OPT_MODEL_PATH = 'mol_optimization.pt'

# --- 重新定义 TransformerRegressor (用于甜度预测) ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

class TransformerRegressor(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, dim_feedforward=256, pad_idx=0):
        super().__init__()
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.regressor = nn.Sequential(nn.Linear(d_model, 64), nn.ReLU(), nn.Dropout(0.1), nn.Linear(64, 1))
    def forward(self, x):
        src_key_padding_mask = (x == self.pad_idx)
        x = self.embedding(x) * math.sqrt(x.size(-1))
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)
        mask_expanded = (~src_key_padding_mask).unsqueeze(-1).float()
        sum_embeddings = torch.sum(x * mask_expanded, dim=1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        feature = sum_embeddings / sum_mask
        return self.regressor(feature).squeeze(-1)

# --- 重新定义 MolSeq2Seq (用于分子优化) ---
class MolSeq2Seq(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, pad_idx=0):
        super().__init__()
        self.d_model = d_model
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoder = nn.Embedding(1024, d_model)
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_layers, num_decoder_layers=num_layers, dim_feedforward=256, batch_first=True, dropout=0.1)
        self.fc_out = nn.Linear(d_model, vocab_size)
    def forward(self, src, tgt):
        src_pad_mask = (src == self.pad_idx)
        tgt_pad_mask = (tgt == self.pad_idx)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1)).to(src.device)
        src_seq = torch.arange(src.size(1), device=src.device).unsqueeze(0).clamp(max=1023)
        tgt_seq = torch.arange(tgt.size(1), device=tgt.device).unsqueeze(0).clamp(max=1023)
        src_emb = self.embedding(src) * math.sqrt(self.d_model) + self.pos_encoder(src_seq)
        tgt_emb = self.embedding(tgt) * math.sqrt(self.d_model) + self.pos_encoder(tgt_seq)
        out = self.transformer(src=src_emb, tgt=tgt_emb, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask, memory_key_padding_mask=src_pad_mask, tgt_mask=tgt_mask)
        return self.fc_out(out)

# ==========================================
# 1. 核心功能函数
# ==========================================
def interactive_optimization(tokenizer):
    print("\n🛠️ 初始化模型中...")
    pad_id = tokenizer.vocab[tokenizer.pad_token]
    sos_id = tokenizer.vocab[tokenizer.sos_token]
    eos_id = tokenizer.vocab[tokenizer.eos_token]

    # 1. 加载优化模型
    opt_model = MolSeq2Seq(len(tokenizer.vocab), pad_idx=pad_id).to(DEVICE)
    try:
        opt_model.load_state_dict(torch.load(OPT_MODEL_PATH, map_location=DEVICE))
        opt_model.eval()
    except FileNotFoundError:
        print(f"❌ 找不到优化模型文件 {OPT_MODEL_PATH}")
        return

    # 2. 加载甜度预测模型
    sweet_model = TransformerRegressor(vocab_size=46, pad_idx=pad_id).to(DEVICE)
    try:
        sweet_model.load_state_dict(torch.load(SWEETNESS_MODEL_PATH, map_location=DEVICE))
        sweet_model.eval()
    except FileNotFoundError:
        print(f"❌ 找不到甜度预测模型文件 {SWEETNESS_MODEL_PATH}")
        return

    # 3. 辅助函数：预测甜度
    def get_pred_score(smiles):
        if not smiles: return 0.0
        ids = tokenizer.encode(smiles)
        inp = torch.tensor([ids], dtype=torch.long).to(DEVICE)
        with torch.no_grad():
            score = sweet_model(inp)
        return score.item()

    # --- 交互循环 ---
    while True:
        print("\n" + "="*50)
        input_smiles = input("🧪 请输入要优化的分子 SMILES (输入 'q' 退出): ").strip()

        if input_smiles.lower() == 'q':
            print("👋 退出程序。")
            break

        # 检查输入合法性
        mol_src = Chem.MolFromSmiles(input_smiles)
        if not mol_src:
            print("❌ 无效的 SMILES，请重新输入。")
            continue

        print(f"🔄 正在尝试优化: {input_smiles}")

        # 准备输入
        src_ids = tokenizer.encode(input_smiles, add_special_tokens=False) + [eos_id]
        src_tensor = torch.tensor([src_ids], dtype=torch.long).to(DEVICE)

        valid_opt_smiles = None
        attempts = 0
        max_attempts = 20 # 最多尝试20次

        # --- 循环生成直到合法 ---
        while attempts < max_attempts:
            attempts += 1
            curr_tgt = torch.tensor([[sos_id]], dtype=torch.long).to(DEVICE)

            with torch.no_grad():
                for _ in range(100): # max len
                    logits = opt_model(src_tensor, curr_tgt)

                    # 使用 Temperature Sampling 增加多样性，避免每次生成一样的死循环
                    logits = logits[:, -1, :] / 1.0
                    probs = F.softmax(logits, dim=-1)
                    next_token = torch.multinomial(probs, 1) # 随机采样

                    if next_token.item() == eos_id:
                        break
                    curr_tgt = torch.cat([curr_tgt, next_token], dim=1)

            pred_ids = curr_tgt[0].cpu().numpy()[1:]
            candidate_smiles = tokenizer.decode(pred_ids, skip_special_tokens=True)

            # 检查是否合法且不是原分子
            if Chem.MolFromSmiles(candidate_smiles) and candidate_smiles != input_smiles:
                valid_opt_smiles = candidate_smiles
                break # 成功找到！

        # --- 结果处理 ---
        if valid_opt_smiles:
            print(f"✅ 优化成功 (尝试次数: {attempts})")
            print(f"   Input:     {input_smiles}")
            print(f"   Optimized: {valid_opt_smiles}")

            # 预测甜度
            score_src = get_pred_score(input_smiles)
            score_opt = get_pred_score(valid_opt_smiles)
            diff = score_opt - score_src

            print(f"   甜度变化: {score_src:.2f} -> {score_opt:.2f} (提升: {diff:+.2f})")

            # 可视化
            mols = [Chem.MolFromSmiles(input_smiles), Chem.MolFromSmiles(valid_opt_smiles)]
            legends = [f"Original\nLogSw: {score_src:.2f}", f"Optimized\nLogSw: {score_opt:.2f}"]
            img = Draw.MolsToGridImage(mols, molsPerRow=2, subImgSize=(300, 300), legends=legends)
            display(img)

        else:
            print(f"⚠️ 优化失败：尝试了 {max_attempts} 次仍未生成有效的优化分子。")
            print("   (模型可能认为原分子已经是局部最优，或者训练数据不足导致无法泛化)")

# --- 启动交互 ---
if __name__ == "__main__":
    if 'tokenizer' in globals():
        interactive_optimization(tokenizer)
    else:
        print("❌ 请先加载 Tokenizer")

## 3. Encoder-Decoder (Seq2Seq):逆合成 (Retrosynthesis)预测

第一部分:数据预处理(Data Preprocessing)

In [ ]:
import pandas as pd
import re
import json
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# ==========================================
# 0. 配置与清洗工具
# ==========================================
BATCH_SIZE = 32
CSV_PATH = 'USPTO_1000.csv' # 请确保文件在当前目录

def remove_atom_mapping(smiles):
    """移除原子映射编号 (例如 [CH2:12] -> [CH2])"""
    return re.sub(r':\d+]', ']', smiles)

def preprocess_uspto(csv_path):
    print("🧹 正在清洗 USPTO 数据...")
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"❌ 错误：找不到 {csv_path}。请确保数据集已上传。")
        return [], []

    products = []
    reactants = []

    # 解析 USPTO 格式: reactants>reagents>production
    col_name = 'reactants>reagents>production'
    if col_name not in df.columns:
        # 尝试寻找可能的列名
        col_name = df.columns[0]

    for rxn in df[col_name]:
        parts = rxn.split('>')
        if len(parts) >= 3:
            r = parts[0] # 反应物
            p = parts[-1] # 产物
            # 清洗
            p = remove_atom_mapping(p)
            r = remove_atom_mapping(r)
            products.append(p)
            reactants.append(r)

    return products, reactants

# ==========================================
# 1. Tokenizer (支持保存/加载)
# ==========================================
class RetrosynthesisTokenizer:
    def __init__(self, vocab=None):
        # 扩展正则
        self.pattern =  r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
        self.regex = re.compile(self.pattern)

        self.pad_token = "<pad>"
        self.unk_token = "<unk>"
        self.sos_token = "<sos>"
        self.eos_token = "<eos>"

        self.special_tokens = [self.pad_token, self.unk_token, self.sos_token, self.eos_token]

        if vocab:
            self.vocab = vocab
        else:
            self.vocab = {t: i for i, t in enumerate(self.special_tokens)}

        self.inv_vocab = {i: t for t, i in self.vocab.items()}

    def tokenize(self, smiles):
        return self.regex.findall(smiles)

    def build_vocab(self, smiles_list):
        print("🏗️ 正在构建逆合成词表...")
        unique_tokens = set()
        for s in tqdm(smiles_list, desc="Scanning Tokens"):
            unique_tokens.update(self.tokenize(s))

        for token in sorted(list(unique_tokens)):
            if token not in self.vocab:
                idx = len(self.vocab)
                self.vocab[token] = idx
                self.inv_vocab[idx] = token
        print(f"✅ 词表构建完成 | Size: {len(self.vocab)}")

    def encode(self, smiles):
        tokens = self.tokenize(smiles)
        return [self.vocab.get(t, self.vocab[self.unk_token]) for t in tokens]

    def decode(self, ids):
        tokens = []
        for i in ids:
            if i == self.vocab[self.eos_token]: break
            if i in [self.vocab[self.sos_token], self.vocab[self.pad_token]]: continue
            tokens.append(self.inv_vocab.get(i, ""))
        return "".join(tokens)

    def save_vocab(self, path):
        with open(path, 'w') as f:
            json.dump(self.vocab, f)

    @classmethod
    def load_vocab(cls, path):
        with open(path, 'r') as f:
            vocab = json.load(f)
        return cls(vocab=vocab)

# ==========================================
# 2. Dataset
# ==========================================
class RetroDataset(Dataset):
    def __init__(self, products, reactants, tokenizer):
        self.data = list(zip(products, reactants))
        self.tokenizer = tokenizer
        self.sos_id = tokenizer.vocab[tokenizer.sos_token]
        self.eos_id = tokenizer.vocab[tokenizer.eos_token]

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        prod, react = self.data[idx]

        # 编码 (注意：不在此处 Padding)
        src_ids = self.tokenizer.encode(prod)
        tgt_ids = self.tokenizer.encode(react)

        return {
            'src': torch.tensor(src_ids + [self.eos_id], dtype=torch.long),
            'tgt_in': torch.tensor([self.sos_id] + tgt_ids, dtype=torch.long),
            'tgt_out': torch.tensor(tgt_ids + [self.eos_id], dtype=torch.long)
        }

def collate_fn(batch):
    pad_id = tokenizer.vocab["<pad>"]
    srcs = pad_sequence([x['src'] for x in batch], batch_first=True, padding_value=pad_id)
    tgt_ins = pad_sequence([x['tgt_in'] for x in batch], batch_first=True, padding_value=pad_id)
    tgt_outs = pad_sequence([x['tgt_out'] for x in batch], batch_first=True, padding_value=pad_id)
    return srcs, tgt_ins, tgt_outs

# --- 执行预处理 ---
if __name__ == "__main__":
    products, reactants = preprocess_uspto(CSV_PATH)

    if products:
        tokenizer = RetrosynthesisTokenizer()
        tokenizer.build_vocab(products + reactants)
        tokenizer.save_vocab("retro_vocab.json")
        print("💾 词表已保存至 retro_vocab.json")

        # 划分数据集
        train_p, test_p, train_r, test_r = train_test_split(products, reactants, test_size=0.1, random_state=42)

        train_dataset = RetroDataset(train_p, train_r, tokenizer)
        test_dataset = RetroDataset(test_p, test_r, tokenizer)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
        # 测试集用于后续独立评估，这里可以先不创建 loader

        # 保存测试数据供后续评估使用
        pd.DataFrame({'Product': test_p, 'Reactant': test_r}).to_csv('retro_test_set.csv', index=False)
        print(f"📊 数据准备就绪: Train={len(train_dataset)}, Test={len(test_dataset)}")

第二部分：模型训练与保存 (Model Training)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import matplotlib.pyplot as plt
from tqdm import tqdm

# ==========================================
# 0. 检查依赖
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if 'tokenizer' not in globals():
    # 尝试加载
    try:
        with open('retro_vocab.json', 'r') as f:
            vocab = json.load(f)
        tokenizer = RetrosynthesisTokenizer(vocab=vocab)
    except:
        raise ValueError("请先运行第一部分代码生成词表！")

# ==========================================
# 1. 模型定义
# ==========================================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

class MolSeq2Seq(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=4, pad_idx=0):
        super().__init__()
        self.d_model = d_model
        self.pad_idx = pad_idx

        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoder = PositionalEncoding(d_model)

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=512,
            batch_first=True,
            dropout=0.1
        )

        self.fc_out = nn.Linear(d_model, vocab_size)

    def create_masks(self, src, tgt):
        src_padding_mask = (src == self.pad_idx)
        tgt_padding_mask = (tgt == self.pad_idx)
        tgt_len = tgt.size(1)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt_len).to(src.device)
        return src_padding_mask, tgt_padding_mask, tgt_mask

    def forward(self, src, tgt):
        src_pad_mask, tgt_pad_mask, tgt_mask = self.create_masks(src, tgt)
        src_emb = self.pos_encoder(self.embedding(src) * math.sqrt(self.d_model))
        tgt_emb = self.pos_encoder(self.embedding(tgt) * math.sqrt(self.d_model))

        output = self.transformer(
            src=src_emb, tgt=tgt_emb,
            src_key_padding_mask=src_pad_mask,
            tgt_key_padding_mask=tgt_pad_mask,
            memory_key_padding_mask=src_pad_mask,
            tgt_mask=tgt_mask
        )
        return self.fc_out(output)

# ==========================================
# 2. 训练流程
# ==========================================
def train_retrosynthesis():
    print("\n🧪 启动逆合成模型训练 (Seq2Seq)...")

    pad_idx = tokenizer.vocab["<pad>"]
    # 增加模型容量以应对逆合成的复杂性
    model = MolSeq2Seq(len(tokenizer.vocab), pad_idx=pad_idx, d_model=32, nhead=2, num_layers=2).to(DEVICE)

    optimizer = optim.Adam(model.parameters(), lr=0.003)
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    losses = []
    epochs = 10

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

        for src, tgt_in, tgt_out in pbar:
            src, tgt_in, tgt_out = src.to(DEVICE), tgt_in.to(DEVICE), tgt_out.to(DEVICE)

            optimizer.zero_grad()
            output = model(src, tgt_in)
            loss = criterion(output.view(-1, output.size(-1)), tgt_out.view(-1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix({'loss': loss.item()})

        avg_loss = total_loss / len(train_loader)
        losses.append(avg_loss)
        scheduler.step(avg_loss)
        print(f"Epoch {epoch+1} Loss: {avg_loss:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}")

    # 保存
    torch.save(model.state_dict(), 'retro_seq2seq.pt')
    print("💾 模型已保存为 retro_seq2seq.pt")

    # 绘图
    plt.figure(figsize=(8, 4))
    plt.plot(losses, label='Train Loss', color='purple')
    plt.title('Retrosynthesis Training Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

if 'train_loader' in globals():
    train_retrosynthesis()
else:
    print("❌ 请先运行第一部分准备数据！")

第三部分：推理、评估与可视化 (Evaluation)

In [ ]:
import torch
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt

# ==========================================
# 0. 配置
# ==========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = 'retro_seq2seq.pt'
TEST_CSV = 'retro_test_set.csv'

# ==========================================
# 1. 核心推理函数
# ==========================================
def predict_reactants(model, product_smiles, tokenizer, max_len=120):
    model.eval()

    # 编码 Product
    src_ids = tokenizer.encode(product_smiles)
    # [SOS] not needed for encoder input in this architecture, just [Product]
    # But encoder doesn't use SOS/EOS usually, decoder does.
    # Let's keep it consistent with training: src was just ids + EOS
    eos_id = tokenizer.vocab[tokenizer.eos_token]
    src = torch.tensor([src_ids + [eos_id]], dtype=torch.long).to(DEVICE)

    # Decoder 输入初始为 [SOS]
    sos_id = tokenizer.vocab[tokenizer.sos_token]
    tgt = torch.tensor([[sos_id]], dtype=torch.long).to(DEVICE)

    generated_ids = []

    with torch.no_grad():
        for _ in range(max_len):
            output = model(src, tgt)
            next_token_logits = output[:, -1, :]
            next_token_id = next_token_logits.argmax(dim=-1).item()

            if next_token_id == eos_id:
                break

            generated_ids.append(next_token_id)
            tgt = torch.cat([tgt, torch.tensor([[next_token_id]], device=DEVICE)], dim=1)

    return tokenizer.decode(generated_ids)

# ==========================================
# 2. 批量评估
# ==========================================
def run_evaluation():
    print("\n🔍 正在评估测试集 (这可能需要几分钟)...")

    # 1. 加载 Tokenizer
    try:
        tokenizer = RetrosynthesisTokenizer.load_vocab("retro_vocab.json")
    except:
        print("❌ 无法加载词表，请检查 retro_vocab.json")
        return

    # 2. 加载模型
    pad_id = tokenizer.vocab["<pad>"]
    model = MolSeq2Seq(len(tokenizer.vocab), pad_idx=pad_id, d_model=32, nhead=2, num_layers=2).to(DEVICE)
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    except FileNotFoundError:
        print(f"❌ 找不到模型文件 {MODEL_PATH}")
        return

    # 3. 加载测试数据
    try:
        df_test = pd.read_csv(TEST_CSV)
    except FileNotFoundError:
        print("❌ 找不到测试集 CSV，请先运行第一部分。")
        return

    results = []
    correct_count = 0

    # 随机取 100 个样本进行快速评估 (全量评估可能太慢)
    sample_size = min(100, len(df_test))
    df_sample = df_test.sample(sample_size, random_state=42)

    for _, row in df_sample.iterrows():
        prod = row['Product']
        true_react = row['Reactant']

        pred_react = predict_reactants(model, prod, tokenizer)

        # 使用 RDKit Canonicalize 进行对比 (忽略写法差异)
        try:
            can_true = Chem.MolToSmiles(Chem.MolFromSmiles(true_react), isomericSmiles=False)
            can_pred = Chem.MolToSmiles(Chem.MolFromSmiles(pred_react), isomericSmiles=False)
            is_match = (can_true == can_pred)
        except:
            is_match = False

        if is_match: correct_count += 1

        results.append({
            'Product': prod,
            'True_Reactants': true_react,
            'Pred_Reactants': pred_react,
            'Match': is_match
        })

    acc = correct_count / sample_size
    print(f"✅ 评估完成 (Sample N={sample_size})")
    print(f"🎯 精确匹配准确率 (Exact Match Accuracy): {acc:.2%}")

    # ==========================================
    # 3. 可视化
    # ==========================================
    print("\n🖼️ 逆合成结果可视化 (Product -> Predicted Reactants)")
    # 挑选 3 个成功的和 3 个失败的
    df_res = pd.DataFrame(results)

    # 尝试混合展示
    to_show = pd.concat([
        df_res[df_res['Match'] == True].head(3),
        df_res[df_res['Match'] == False].head(3)
    ])

    mols = []
    legends = []

    for _, row in to_show.iterrows():
        m_prod = Chem.MolFromSmiles(row['Product'])
        m_pred = Chem.MolFromSmiles(row['Pred_Reactants'])

        if m_prod and m_pred:
            mols.extend([m_prod, m_pred])
            status = "✅ Correct" if row['Match'] else "❌ Wrong"
            legends.extend(["Product", f"Pred Reactants\n({status})"])

    if mols:
        img = Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(300, 300), legends=legends)
        display(img)

if __name__ == "__main__":
    run_evaluation()

第四部分：单分子交互推理 (User Input)

In [ ]:
import torch
from rdkit import Chem
from rdkit.Chem import Draw

# ==========================================
# 交互式工具
# ==========================================
def interactive_retrosynthesis():
    print("\n🧪 交互式逆合成分析工具")
    print("=========================")

    # 1. 资源加载
    try:
        tokenizer = RetrosynthesisTokenizer.load_vocab("retro_vocab.json")
        pad_id = tokenizer.vocab["<pad>"]
        model = MolSeq2Seq(len(tokenizer.vocab), pad_idx=pad_id, d_model=32, nhead=2, num_layers=2).to(DEVICE)
        model.load_state_dict(torch.load('retro_seq2seq.pt', map_location=DEVICE))
        model.eval()
        print("✅ 模型已加载 Ready.")
    except Exception as e:
        print(f"❌ 初始化失败: {e}")
        return

    # 2. 推理辅助函数
    def infer(smiles):
        # 简单预处理
        smiles = re.sub(r':\d+]', ']', smiles)

        src_ids = tokenizer.encode(smiles)
        eos_id = tokenizer.vocab["<eos>"]
        src = torch.tensor([src_ids + [eos_id]], dtype=torch.long).to(DEVICE)

        sos_id = tokenizer.vocab["<sos>"]
        tgt = torch.tensor([[sos_id]], dtype=torch.long).to(DEVICE)

        res_ids = []
        with torch.no_grad():
            for _ in range(150):
                out = model(src, tgt)
                next_id = out[:, -1, :].argmax().item()
                if next_id == eos_id: break
                res_ids.append(next_id)
                tgt = torch.cat([tgt, torch.tensor([[next_id]], device=DEVICE)], dim=1)

        return tokenizer.decode(res_ids)

    # 3. 循环
    while True:
        print("\n" + "-"*60)
        user_input = input("请输入目标产物 SMILES (输入 'q' 退出): ").strip()

        if user_input.lower() == 'q':
            print("👋 Bye!")
            break

        if not Chem.MolFromSmiles(user_input):
            print("⚠️ 无效的 SMILES，请检查输入。")
            continue

        print(f"🔄 正在分析逆合成路径...")
        try:
            pred_reactants = infer(user_input)
            print(f"✅ 预测反应物: {pred_reactants}")

            # 可视化
            m_prod = Chem.MolFromSmiles(user_input)
            m_react = Chem.MolFromSmiles(pred_reactants)

            if m_react:
                print("🖼️ 生成反应图示:")
                img = Draw.MolsToGridImage(
                    [m_prod, m_react],
                    molsPerRow=2,
                    subImgSize=(400, 300),
                    legends=["Target Product", "Predicted Reactants"]
                )
                display(img)
            else:
                print("⚠️ 预测结果不是有效的分子结构。")

        except Exception as e:
            print(f"❌ 推理过程出错: {e}")

# 启动
if __name__ == "__main__":
    interactive_retrosynthesis()